## Libraries

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate, Activation, Concatenate
from tensorflow.keras.metrics import IoU, BinaryIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
import cv2 as cv
from tensorflow.keras.applications import VGG16

## Loading Data

In [2]:
home = os.environ['HOME']

In [3]:
#Path for entire dataset

#path_X = os.path.join(home,'raw_data/image_slices')
#path_y = os.path.join(home,'raw_data/mask_slices')

In [1]:
#Path for subset of dataset

path_small_X = os.path.join(home,'raw_data/small_dataset/sample_images')
path_small_y = os.path.join(home,'raw_data/small_dataset/sample_masks')

NameError: name 'os' is not defined

In [4]:
split_ratio = 0.9

In [5]:
def train_val_split (path_X, path_y, split_ratio):
    X_names = os.listdir(path_X)
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    X_path = [f'{path_X}/{file}' for file in X_names]
    train_X, val_X = X_path[:int(len(X_path)*split_ratio)], X_path[int(len(X_path)*split_ratio):]
    train_y, val_y = y_path[:int(len(y_path)*split_ratio)], y_path[int(len(y_path)*split_ratio):]
    return train_X, val_X, train_y, val_y 

In [ ]:
#Small Dataset
train_X, val_X, train_y, val_y = train_val_split (path_small_X, path_small_y, split_ratio)

In [6]:
#Entire Dataset
# train_X, val_X, train_y, val_y = train_val_split (path_X, path_y, split_ratio)

In [7]:
def verify_matching_input_labels(X_names, y_names):
    for x, y in zip(X_names, y_names):
        if os.path.basename(x) != os.path.basename(y):
            raise ValueError(f"X and Y not matching: {x, y}")

In [8]:
verify_matching_input_labels(train_X, train_y)

In [9]:
verify_matching_input_labels(val_X, val_y)

In [10]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1) / 255 
    return image, mask

In [11]:
def batch_data (X_path, y_path, batch_size):
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.shuffle(buffer_size = len(X_path), seed = 10).map(process_path).batch(batch_size)

### Training Dataset

In [12]:
train_dataset = batch_data(train_X, train_y, batch_size=16)

2022-12-01 13:32:17.444137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 13:32:17.581318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 13:32:17.583166: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 13:32:17.587356: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

### Validation Dataset

In [13]:
val_dataset = batch_data(val_X, val_y, batch_size=16)

### Test Dataset

In [14]:
# path_X_TEST = os.path.join(home,'raw_data/TEST_slices/test_image_slices')
# path_y_TEST = os.path.join(home,'raw_data/TEST_slices/test_mask_slices')

In [15]:
def batch_data_test (path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size)

In [16]:
#TEST_dataset = batch_data_test(path_X_TEST, path_y_TEST, batch_size=16)

## Transfer Learning Model Definition

In [5]:
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [17]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [19]:
def decoder_block(inputs, skip_features, num_filters): #skip features are going to be the x returned from the encoder block
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [ ]:
def build_vgg16_unet(img_height, img_width, channels):
    inputs = Input((img_height, img_width, channels))
    
    vgg16= VGG16(include_top=False, weights='imagenet', input_tensor=inputs, )
    
    #Encoder
    skip1 = vgg16.get_layer("block1_conv2").output #shape 256, filters 64
    skip2 = vgg16.get_layer("block2_conv2").output #shape 128, filters 128
    skip3 = vgg16.get_layer("block3_conv3").output #shape 64, filters 256
    skip4 = vgg16.get_layer("block4_conv3").output #shape 32, filters 512
        
    #Bottleneck/Bridge ofthe Unet
    b1 = vgg16.get_layer("block5_conv3").output #shape 16, filters 512
    
    #Decoder
    decoder1 = decoder_block(b1, skip4, 512)
    decoder2 = decoder_block(decoder1, skip3, 256)
    decoder3 = decoder_block(decoder2, skip2, 128)
    decoder4 = decoder_block(decoder3, skip1, 64)
    
    #Output
    outputs = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(decoder4)
    model = Model(inputs, outputs)
    
    iou = BinaryIoU()
    
    model.compile(optimizer='adam', loss=loss_sum, metrics=['accuracy', iou], name='VGG16_Unet')
    
    model.summary()
    
    return model


In [20]:
def dice_loss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(targets * inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

In [21]:
def loss_sum(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    o = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred) + dice_loss(y_true, y_pred)
    return tf.reduce_mean(o)

In [23]:
model = build_vgg16_unet(256, 256, 3)

In [24]:
checkpoint_filepath = '../tmp/vgg_unet/version1'
es = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', restore_best_weights=True)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs = 500, callbacks=[es, checkpoint], verbose=1)

Epoch 1/500


2022-12-01 13:32:24.247942: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


 523/3845 [===>..........................] - ETA: 8:36 - loss: 0.7240 - accuracy: 0.8867 - binary_io_u: 0.6829